# Z-score 
* 상품군_표준편차 = 0 : 없음
* 상품군&브랜드_표준편차 = 0 : 1895 (결측값 또한 존재)
* NEW_표준편차 = 0 : 18078 (상품 가격이 동일한 경우)
* **0인 경우 z-score 또한 0 값으로 만들어 주기** 
* **결측값은 NaN 으로 유지** 

In [1]:
import os
import sys
import joblib

import pandas as pd 
import numpy as np
pd.options.display.max_rows = 100
pd.options.display.max_columns = 500
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import imp
import datetime
from tqdm import tqdm

In [18]:
import os
import sys
import math
import joblib
import numpy as np
import pandas as pd
import json
import requests
from datetime import datetime, timedelta

In [25]:
sale2 = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', "sale_data_v05_0828.xlsx"))

In [19]:
## -- JB -- (수정필요??)
def engineering_TimeDiff(df) :
    
    # [상품 및 브랜드 방송 노출 횟수] 동일 상품 / 브랜드 총 방송횟수 - 먼저 돌리기 !!
#     item_count = df.dropna(subset = ["노출(분)"]).groupby('NEW상품명').count()['방송일시'].reset_index().rename(columns = {'방송일시' : '상품노출횟수'})
#     brand_count = df.dropna(subset = ["노출(분)"]).groupby('브랜드').count()['방송일시'].reset_index().rename(columns = {'방송일시' : '브랜드노출횟수'})
#     df = df.merge(item_count, on = 'NEW상품명', how = 'left')
#     df = df.merge(brand_count, on = '브랜드', how = 'left')

    # [동일상품 방송 시간차] 동일 상품 별 시간 간격
    df['방송일'] = df['방송일시'].dt.date
    t = df.groupby(['NEW상품명','방송일'])['취급액'].sum().reset_index()[['NEW상품명','방송일']]
    total = []
    for item in t['NEW상품명'].unique():
        timediff = t.loc[t['NEW상품명'] == item, '방송일']
        a = pd.DataFrame(list(zip(timediff, timediff.diff())))
        a['NEW상품명'] = item
        total.extend(a.values)
    timediff = pd.DataFrame(total, columns = ['방송일', '방송시간차', 'NEW상품명'])
    timediff.loc[timediff['방송시간차'].isnull(), '방송시간차'] = 0
    timediff['방송시간차'] = timediff['방송시간차'].apply(lambda x : x.days if x != 0 else x)
    df = df.merge(timediff, on = ['방송일', 'NEW상품명'], how = 'left')
    
    # [상품 및 브랜드 총 판매 횟수 ] 동일 상품 / 브랜드 총 방송횟수
    df = df.merge(df.groupby('NEW상품명')['방송일'].nunique().reset_index().rename(columns = {'방송일' : '상품방송횟수'}), on = 'NEW상품명', how = 'left')
    
    df = df.merge(df.groupby('브랜드')['방송일'].nunique().reset_index().rename(columns = {'방송일' : '브랜드방송횟수'}), on = '브랜드', how = 'left')
    
    df = df.drop('방송일', axis = 1)
    
    return df

## -- YJ --

def engineering_Soldout(df):
    # [조기매진] (20분 이하 혹은 20분과 30분 사이에 조기 종료된 프로그램 선별)
    df['조기매진'] = df['노출(분)'].map(lambda x: 1 if ((x < 20) | (x > 20) & (x < 30)) else 0)     # 20분 이하, 20-30 분 사이
    # 방송일시의 nan 값 채워주는 부분
    df["노출(분)"] = df["노출(분)"].fillna(method='ffill')
    return df

## -- JS --

def engineering_DatePrice(df):
    # 데이터 로드
    item = pd.read_excel(os.path.join('..', '..','0.Data', '01_제공데이터', 'item_meta_v03_0823.xlsx'))
    data = joblib.load(os.path.join('..', '..','0.Data', '01_제공데이터', '0823_prep4data.pkl'))
    itemcategory = data['itemcategory']
    mothercode = data['mothercode']
    brand = data['brand']
    
    # 공휴일여부
    key = '8wBiTSHPiK2z%2By8ETu%2FpYv%2FMAAdZoR8rZg3PIvSNCcD%2F26BiBPaosFs2dzrVJ%2BHUeaQGWb9c3T4vvNgMpI7fdw%3D%3D'
    def getHoliday(year):
        url = f'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&ServiceKey={key}&_type=json&numOfRows=20'
        response = requests.get(url)
        holidays = response.json()['response']['body']['items']['item']
        holidays = pd.DataFrame(holidays)
        holidays['locdate'] = holidays['locdate'].astype(str).apply(lambda x : '-'.join([x[:4], x[4:6], x[6:]]))
        return holidays
    
    # 무형 상품군 NEW상품명 채워주기(무형 상품군은 어짜피 나중에 버릴것)
    df.loc[df['NEW상품명'].isnull(), 'NEW상품명'] = df.loc[df['NEW상품명'].isnull(), '상품명']
    
    ## [공휴일여부]
    print('연도입력')
    year = input()
    holidays = getHoliday(year)
    df = df.merge(holidays[['locdate', 'isHoliday']], left_on = df['방송일시'].dt.date.astype(str), right_on = 'locdate', how = 'left').drop('locdate', axis = 1)
    df['isHoliday'] = df['isHoliday'].apply(lambda x : 1 if x == 'Y' else 0)
    ## [날짜 및 시간대]
    df['방송월'] = df['방송일시'].dt.month
    df['방송일'] = df['방송일시'].dt.day
    df['방송시간(시간)'] = df['방송일시'].dt.hour.apply(lambda x : 24 if x == 0 else x)
    df['방송시간(분)'] = df['방송일시'].dt.minute
    
    
    ## [평일여부] (평일 : 0, 주말 : 1)
    df['평일여부'] = df['방송일시'].dt.weekday.apply(lambda x : 0 if x < 5 else 1)
    
    ## [방송시간대] (아침/오전/오후/저녁/밤 (아침(6:00~9:00)/오전(9:00~12:00)/오후(12:00~18:00)/저녁(18:00~22:00)/밤(22:00~2:00))
    df['방송시간대'] = df['방송일시'].dt.hour.apply(lambda x : '아침' if 5 < x <= 9 else 
                                          ('오전' if 10 <= x <= 12 else
                                           ( '오후' if 13 <= x <= 18 else
                                            ('저녁' if 19 <= x <= 22 else
                                             ('밤' if 23 <= x < 3 else x)
                                            ))))
    
    ## [계절] (봄(3~5), 여름6~8), 가을(9~11), 겨울(12~2))
    df['계절'] = df['방송월'].apply(lambda x : '봄' if 3 <= x <= 5 else
                            ('여름' if 6 <= x <= 8 else
                             ('가을' if 9 <= x <= 11 else 
                              ('겨울' if x < 3 or x > 11 else x))))
    
    ## [분기] 
    df['분기'] = df['방송월'].apply(lambda x : '1분기' if 1 <= x <= 3 else
                            ('2분기' if 4 <= x <= 6 else
                             ('3분기' if 7 <= x <= 9 else 
                              ('4분기' if 10 <= x <= 12 else x))))
    
    ## [성별] (공용 : 0, 남성 : 1, 여성 : 2)
    df['성별'] = df['NEW상품명'].apply(lambda x : 1 if '남성' in x else (2 if '여성' in x else 0))
    
    ## [상품군]별 가격 summary
    df = df.merge(itemcategory, on = '상품군', how = 'left')
    
    ## [마더코드]별 가격 summary
    df = df.merge(mothercode, on = '마더코드', how = 'left')
    
    ## [브랜드]별 가격 summary
    df = df.merge(brand, on = ['브랜드', '상품군'], how = 'left')
    
    ## [NEW아이템]별 가격 summary + [상품군] 별 가격대, 전체 가격대
    df = df.merge(item, on = ['NEW상품코드', 'NEW상품명', '상품군'], how = 'left')
    
    ## [상품군] 평균 판매단가 - 해당 상품 판매단가
    df['상품군평균판매단가차이'] = df['상품군_평균판매단가'] - df['판매단가']
    
    # [결합상품여부]
    df['결합상품'] = df['NEW상품명'].apply(lambda x : 1 if '+' in x else 0)
    
    return df


def engineering_trendnorder(df):
    # 방송날짜별 상품군별 취급액 계산
    sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))
    sale['방송날짜'] = sale['방송일시'].dt.date
    temp = sale.groupby(['상품군', '방송날짜'])['취급액'].sum().reset_index()
    temp['방송날짜'] = pd.to_datetime(temp['방송날짜'])
    
    df['log최근3개월상품군추세'] = None
    
    # 방송날짜 90일 전부터 방송날짜 전까지의 취급액 log 추세 구하기
    df['방송날짜'] = df['방송일시'].dt.date
    df['방송날짜'] = pd.to_datetime(df['방송날짜'])
    for cate, date in df[['상품군', '방송날짜']].drop_duplicates().values:
        log_y = np.log(list(temp.loc[(temp['상품군'] == cate) & (temp['방송날짜'] < date) & (date - timedelta(days = 90) < temp['방송날짜']), '취급액'] + 1))
        log_x = np.arange(len(log_y))
        try:
            log_z = np.polyfit(log_x, log_y, 1)[0]
        except:
            log_z = 0

        df.loc[(df['상품군'] == cate) & (df['방송날짜'] == date), 'log최근3개월상품군추세'] = log_z
    
    
    sale['방송월'] = sale['방송일시'].dt.month
    sale['방송시간(시간)'] = sale['방송일시'].dt.hour
    sale['방송시간(분)'] = sale['방송일시'].dt.minute
    sale['판매량'] = sale['취급액'] / sale['판매단가']
    sale['판매량'] = sale['판매량'].fillna(0).apply(lambda x : math.ceil(x))
    
    # 월별 상품군 판매량
    temp = pd.pivot_table(sale, index = '상품군', columns = '방송월', values = '판매량', aggfunc = np.sum).T.reset_index()
    temp.columns = [temp.columns[0]] + list(map(lambda x : '월별판매랑_' + x, temp.columns[1:]))
    df = df.merge(temp, on = '방송월', how = 'left')
    
    # 시간대별 상품군 판매량
    temp = pd.pivot_table(sale, index = '상품군', columns = '방송시간(시간)', values = '판매량', aggfunc = np.sum).T.reset_index()
    temp.columns = [temp.columns[0]] + list(map(lambda x : '시간대별판매랑_' + x, temp.columns[1:]))
    df = df.merge(temp, on = '방송시간(시간)', how = 'left')
    
    # 시간별 상품군 판매량
    temp = pd.pivot_table(sale, index = '상품군', columns = '방송시간(분)', values = '판매량', aggfunc = np.mean).T.reset_index()
    temp.columns = [temp.columns[0]] + list(map(lambda x : '시간별판매랑_' + x, temp.columns[1:]))
    df = df.merge(temp, on = '방송시간(분)', how = 'left')
    
    # 할인율
    rt = pd.read_csv(os.path.join('..', '..', '0.Data', '01_제공데이터', 'prep_discountRt.csv'), encoding = 'cp949')
    df['할인율'] = rt.values
    
    return df

In [26]:
sale2 = engineering_TimeDiff(sale2)

In [27]:
sale2 = engineering_Soldout(sale2)

In [30]:
sale2 = engineering_DatePrice(sale2)

연도입력


 2019


In [39]:
sale2.shape

(38309, 63)

In [42]:
sale3 = sale2.copy()

In [43]:
def engineering_zscore(df):
#     brand = df.columns[df.columns.str.contains("상품군&브랜드")]
#     for b in brand:
#         df[b] = df[b].fillna(0)

    def zscore(price, mean, std):
        if std == 0:
            return 0
        else:
            return (price - mean) / std

    df["상품군_zscore"] = df.apply(lambda x: zscore(x["판매단가"], x["상품군_평균판매단가"], x["상품군_표준편차"]), axis=1)
    df["상품군&브랜드_zscore"] = df.apply(lambda x: zscore(x["판매단가"], x["상품군&브랜드_평균판매단가"], x["상품군&브랜드_표준편차"]), axis=1)
    df["마더코드_zscore"] = df.apply(lambda x: zscore(x["판매단가"], x["마더코드_평균판매단가"], x["마더코드_표준편차"]), axis=1)
    df["NEW_zscore"] = df.apply(lambda x: zscore(x["판매단가"], x["NEW_평균판매단가"], x["NEW_표준편차"]), axis=1)

    # z-score 만들어 줬으니 평균, 분산, 표준편차 제거!
    stat_mean = df.columns[df.columns.str.contains("_평균판매단가")]
    stat_var = df.columns[df.columns.str.contains("분산")]
    stat_std = df.columns[df.columns.str.contains("표준편차")]

    stats = [stat_mean, stat_var, stat_std]
    for stat in stats:
        df.drop(stat, axis=1, inplace=True)

    return df

In [44]:
sale3 = engineering_zscore(sale3)

In [45]:
sale3.shape

(38309, 55)

In [46]:
sale3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38309 entries, 0 to 38308
Data columns (total 55 columns):
방송일시              38309 non-null datetime64[ns]
노출(분)             38309 non-null float64
마더코드              38309 non-null int64
상품코드              38309 non-null int64
상품명               38309 non-null object
상품군               38309 non-null object
판매단가              38309 non-null int64
취급액               35379 non-null float64
NEW상품코드           37372 non-null float64
NEW상품명            38309 non-null object
결제방법              37372 non-null float64
옵션                4140 non-null object
종류                22744 non-null object
브랜드               33066 non-null object
상품명다시             37372 non-null object
단위                22390 non-null object
방송시간차             37372 non-null float64
상품방송횟수            37372 non-null float64
브랜드방송횟수           33066 non-null float64
조기매진              38309 non-null int64
log최근3개월상품군추세     25953 non-null object
방송날짜              38309 non-null datetime